## Custom Resource Management: Writing Context Managers
- Whenever you need custom setup/teardown logic, you can write your own Context Manager.
- A context manager ensures that teardown always runs, even if errors occur in the block.  
- Two approaches: implement `__enter__`/`__exit__` in a class or use the simpler generator-based decorator.  

In [7]:
class MyContextManager:
    def __init__(self, timeout):
        self.timeout = timeout

    def __enter__(self):
        print("Setup complete")
        return "a simple value"

    #def __exit__(self, exception_type, exception_value, traceback):
    def __exit__(self, *args):
        print(f"Teardown complete")

        for arg in args:
            print(arg )

        #return False  # Return True to suppress exceptions
        return True  # Return False to propagate exceptions
    

with MyContextManager(timeout=5) as cm:
    print(f"{cm}")
    print("Inside the block")
    raise ValueError("Simulated error")

Setup complete
a simple value
Inside the block
Teardown complete
<class 'ValueError'>
Simulated error


## The `@contextlib.contextmanager` Decorator
- Provided by the `contextlib` module to turn a generator into a context manager.  
- Decorated function needs exactly one `yield`.  
- Code before `yield` runs as `__enter__`; code after (or in `finally`) runs as `__exit__`.  
- Simplifies many common patterns without writing a full class.

###  Generator Structure for `@contextmanager`
- Wrap the `yield` in `try...finally` to ensure teardown even on errors.  
- The value yielded is bound to `as var` in the `with` statement (if used).  
- You can catch exceptions inside the generator if you want to suppress them.

In [8]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(destination):
    """
    Temporarily switch into destination. If the directory does not exists, 
    it is created just before the switch.

    Args:
        destination (str): Path to the directory that should become the working directory.
    """
    origin_directory = os.getcwd()

    try:
        print(f"Changing into directory: {destination}")    
        os.makedirs(destination, exist_ok=True)
        os.chdir(destination)
        yield os.getcwd()
    finally:
        print(f"Returning to original directory: {origin_directory}")
        os.chdir(origin_directory)

print(f"Start: {os.getcwd()}")   

with change_directory("temp_directory") as new_directory:
    print(f"Inside : {new_directory}")

print(f"End: {os.getcwd()}")    

Start: C:\Users\Shubhesh Swain\Desktop\DevOps\Udemy\python-devops-start
Changing into directory: temp_directory
Inside : C:\Users\Shubhesh Swain\Desktop\DevOps\Udemy\python-devops-start\temp_directory
Returning to original directory: C:\Users\Shubhesh Swain\Desktop\DevOps\Udemy\python-devops-start
End: C:\Users\Shubhesh Swain\Desktop\DevOps\Udemy\python-devops-start
